# 프로젝트 : 단어 Level로 번역기 업그레이드하기

## 루브릭

1. 번역기 모델 학습에 필요한 텍스트 데이터 전처리가 잘 이루어졌다.	
 - 구두점, 대소문자, 띄어쓰기 등 번역기 모델에 요구되는 전처리가 정상적으로 진행되었다.
2. seq2seq 기반의 번역기 모델이 정상적으로 구동된다.	
 - seq2seq 모델 훈련결과를 그래프로 출력해보고, validation loss그래프가 우하향하는 경향성을 보이며 학습이 진행됨이 확인되었다.
3. 테스트 결과 의미가 통하는 수준의 번역문이 생성되었다.	
 - 테스트용 디코더 모델이 정상적으로 만들어졌으며, input(영어)와 output(프랑스어) 모두 한글로 번역해서 결과를 출력해보았고, 둘의 내용이 유사함을 확인하였다.


In [1]:
import tensorflow
import pandas as pd
import tensorflow.keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np
import os

print(tensorflow.__version__)

2.6.0


In [2]:
file_path = os.getenv('HOME')+'/aiffel/translator_seq2seq/data/fra.txt'
lines = pd.read_csv(file_path, names=['eng', 'fra', 'cc'], sep='\t')
print('전체 샘플의 수 :',len(lines))
lines.sample(5)

전체 샘플의 수 : 217975


,eng,fra,cc
59246,What happened exactly?,"Que s'est-il produit, exactement ?",CC-BY 2.0 (France) Attribution: tatoeba.org #1...
175323,I can't forget about that stupid movie.,Je n'arrive pas à oublier ce film stupide.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
27942,Something's wrong.,Quelque chose ne va pas.,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
167775,Let's sit down and discuss it calmly.,Asseyons-nous et discutons-en calmement.,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
65332,"That one's for me, too.",Celui-là me revient également.,CC-BY 2.0 (France) Attribution: tatoeba.org #3...


In [3]:
lines = lines[['eng', 'fra']][:33000] # 글자 단위가 아닌 단어 단위는 단어장의 크기가 커지고 학습속도가 느려지기에 33000개의 샘플만 사용
lines.sample(5)

,eng,fra
4999,I called you.,Je t'ai appelé.
18101,Who called them?,Qui les a appelés ?
29939,Who believes that?,Qui croit ça ?
17655,We all hate Tom.,Nous détestons tous Tom.
32669,I love butterflies.,J'adore les papillons.


## Step 1. 정제, 정규화, 전처리 (영어, 프랑스어 모두!)

### 1. 구두점(Punctuation)을 단어와 분리

In [4]:
import re

pattern = r'\w+|[^\w\s]'

l_eng = lines.eng
for i, text in enumerate(l_eng):
    
    result = re.findall(pattern, text)
    l_eng[i] = ' '.join(result)
    
l_eng.sample(5)

14367       Expect no mercy .
20539      I was born there .
6521           We have some .
31630    He entered my room .
6162           They said no .
Name: eng, dtype: object

In [5]:
l_fra = lines.fra
for i, text in enumerate(l_fra):
    
    result = re.findall(pattern, text)
    l_fra[i] = ' '.join(result)
    
l_fra.sample(5)

26757              Je suis sérieuse .
8343              Ça a l ' air bien .
31490    Donnez - moi votre couteau .
4265              On s ' est perdus .
13042       C ' est ce que veut Tom .
Name: fra, dtype: object

- 're'모듈을 사용하여 pattern 정규표현식을 이용하여 구두점과 단어 분리하였다.

### 2. 소문자로 변환

In [6]:
l_eng = l_eng.apply(lambda x: x.lower()) 
l_fra= l_fra.apply(lambda x: x.lower()) 

l_eng.sample(5)
l_fra.sample(5)

28498                      elles haletaient .
19183            accordez - moi une seconde !
31151    voulez - vous que je vous véhicule ?
14032     sont - ils en mesure de nous voir ?
1904                           vers le nord .
Name: fra, dtype: object

- lower()를 사용하여 간단하게 소문자로 변환하였다.

### 3. 디코더의 문장에 시작 토큰과 종료 토큰 삽입

In [7]:
sos_token = 'sos'
eos_token = 'eos'

In [8]:
l_fra = l_fra.apply(lambda x : '<sos> '+ x + ' <eos>')

l_fra.sample(5)

20927    <sos> j ' en ai marre du poisson ! <eos>
31080           <sos> aimez - vous boston ? <eos>
27275          <sos> ce n ' est pas grave . <eos>
27935             <sos> quelqu ' un parle . <eos>
28118          <sos> c ' est bon pour moi . <eos>
Name: fra, dtype: object

### 4. 띄어쓰기 단위로 토큰화

In [9]:
eng_tokenizer = Tokenizer(char_level=False)
eng_tokenizer.fit_on_texts(l_eng)              
input_text = eng_tokenizer.texts_to_sequences(l_eng)
input_text[:3]

[[25], [25], [25]]

In [10]:
fra_tokenizer = Tokenizer(char_level=False)  
fra_tokenizer.fit_on_texts(l_fra)
target_text = fra_tokenizer.texts_to_sequences(l_fra)

target_text[:3]

[[1, 67, 2], [1, 313, 2], [1, 22, 494, 2]]

- Tokenizer(char_level=True)를 하면 문자 단위로 토큰화
- Tokenizer(char_level=False)를 하면 단어 단위로 토큰화

## Step 3. 케라스의 토크나이저로 텍스트를 숫자로

In [11]:
eng_vocab_size = len(eng_tokenizer.word_index) + 1   # 0번 토큰을 고려하여 +1
fra_vocab_size = len(fra_tokenizer.word_index) + 1
max_eng_seq_len = max([len(line) for line in input_text])
max_fra_seq_len = max([len(line) for line in target_text])
print('전체 샘플의 수 :',len(lines))
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

전체 샘플의 수 : 33000
영어 단어장의 크기 : 4544
프랑스어 단어장의 크기 : 8303
영어 시퀀스의 최대 길이 8
프랑스어 시퀀스의 최대 길이 17


In [12]:
encoder_input = input_text
# 종료 토큰 제거
decoder_input = [[char for char in line if char != fra_tokenizer.word_index[eos_token]] for line in target_text] 

# 시작 토큰 제거
decoder_target = [[char for char in line if char != fra_tokenizer.word_index[sos_token]] for line in target_text]

In [13]:
print(decoder_input[:3])
print(decoder_target[:3])

[[1, 67], [1, 313], [1, 22, 494]]
[[67, 2], [313, 2], [22, 494, 2]]


In [14]:
encoder_input = pad_sequences(encoder_input, maxlen = max_eng_seq_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen = max_fra_seq_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen = max_fra_seq_len, padding='post')
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

In [15]:
n_of_val = 3000

encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

print('영어 학습데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 학습 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 학습 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 학습데이터의 크기(shape) : (33000, 8, 4544)
프랑스어 학습 입력데이터의 크기(shape) : (33000, 17, 8303)
프랑스어 학습 출력데이터의 크기(shape) : (33000, 17, 8303)


## Step 4. 임베딩 층(Embedding layer) 사용하기

In [16]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model

print(eng_vocab_size)

4544


In [17]:
# 입력 텐서 생성.
encoder_inputs = Input(shape=(None, ))
encoder_emb = Embedding(eng_vocab_size, 64)(encoder_inputs)
encoder_lstm = LSTM(units = 64, return_state = True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_emb)
encoder_states = [state_h, state_c]

In [18]:
# 입력 텐서 생성.

decoder_inputs = Input(shape=(None,))
decoder_emb = Embedding(fra_vocab_size, 64)(decoder_inputs) 
decoder_lstm = LSTM(units = 64, return_sequences = True, return_state=True)
decoder_outputs, _, _= decoder_lstm(decoder_emb, initial_state = encoder_states)


In [19]:
decoder_softmax_layer = Dense(fra_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

In [20]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 64)     290816      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 64)     531392      input_2[0][0]                    
______________________________________________________________________________________________

In [ ]:
history = model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train,
          validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size=16, epochs=20)

In [ ]:
%matplotlib inline
plt.figure(figsize=(15,4))
epochs = range(1, len(history.history['accuracy']) + 1)

# loss 그래프
plt.subplot(1,2,1)
plt.plot(epochs, history.history['loss'], 'r', label = 'train loss')
plt.plot(epochs, history.history['val_loss'], 'b', label='val loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()

# accuracy 그래프
plt.subplot(1,2,2)
plt.plot(epochs, history.history['accuracy'], 'r', label='train accuracy')
plt.plot(epochs, history.history['val_accuracy'], 'b', label='val accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()

plt.show()

## Step 5. 모델 구현하기

In [ ]:
encoder_model = Model(inputs = encoder_inputs, outputs = encoder_states)
encoder_model.summary()

In [ ]:
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_emb = Embedding(fra_vocab_size, 64)(decoder_inputs) 

decoder_outputs, state_h, state_c = decoder_lstm(decoder_emb, initial_state = decoder_states_inputs)
decoder_states = [state_h, state_c]

In [ ]:
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)
decoder_model.summary()

In [ ]:
eng2idx = eng_tokenizer.word_index
fra2idx = fra_tokenizer.word_index
idx2eng = eng_tokenizer.index_word
idx2fra = fra_tokenizer.index_word

In [ ]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1, fra_vocab_size))
    target_seq[0, 0, fra2idx['\t']] = 1.

    stop_condition = False
    decoded_sentence = ""

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = idx2fra[sampled_token_index]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += sampled_char

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_fra_seq_len):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1, fra_vocab_size))
        target_seq[0, 0, sampled_token_index] = 1.

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

## Step 6. 모델 평가하기

In [ ]:
for seq_index in [10,200,30,40,501]: # 입력 문장의 인덱스 (자유롭게 선택해 보세요)
    input_seq = encoder_input[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', lines.eng[seq_index])
    print('정답 문장:', lines.fra[seq_index][1:len(lines.fra[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)-1]) # '\n'을 빼고 출력